# Create photoreduction forcing files

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.basemap import Basemap, cm
import cmocean
import netCDF4 as nc
import datetime as dt
import pickle

%matplotlib notebook

Parameterization for photo reduction:
- Higher sol_flux means stronger photo reduction.
- Background reduction value: 1
- Photo-enhanced reduction value maximum: 58

##### Load files

In [4]:
mask  = nc.Dataset('/ocean/brogalla/GEOTRACES/ariane_runs/ANHA12_Ariane_mesh.nc')
tmask = np.array(mask.variables['tmask'])
cond = (tmask[0,:,:,:] > 0.1) 
land_mask = np.ma.masked_where(cond, tmask[0,:,:,:])

mesh  = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12-EXH006_5d_gridT_y2002m01d05.nc')
lons  = np.array(mesh.variables['nav_lon'])
lats  = np.array(mesh.variables['nav_lat'])
tmask = tmask[0,:,imin:imax,jmin:jmax]

##### Functions

In [12]:
def load_variables(filename):
    a = nc.Dataset(filename)
    
    sol_flux = np.array(a.variables['iocesflx'][:,0,:,:])     
    
    return sol_flux

In [27]:
def red_rate_calc(year):
    
    imin, imax = 1480, 2180
    jmin, jmax = 160, 800
    
    sol_flux = load_variables('/data/brogalla/ANHA12/icemod_ave/ANHA12_EXH006_'+str(year)+'_monthly.nc')
    red_rate = (np.abs((sol_flux - \
                        sol_flux[:,imin:imax,jmin:jmax].mean()))/sol_flux[:,imin:imax,jmin:jmax].max())*58
    red_rate[red_rate < 1] = 1
    
    return red_rate

In [28]:
def save_file(filename, field1):
    ncd = nc.Dataset(filename, 'w', zlib=True)

    ncd.createDimension('x',len(mesh.dimensions['x']))
    ncd.createDimension('y',len(mesh.dimensions['y']))
    ncd.createDimension('time_counter',None)
    
    # variables
    red             = ncd.createVariable('kred', 'float64', ('y','x'))
    red.units       = 'Unitless'
    red.long_name   = 'Photoreduction multiplier'  
    red.coordinates = 'nav_lon nav_lat'
    red[:]          = field1[:]

    ncd.close()
    return

In [29]:
def red_year(year, savefile=False):
    
    red_rateyear = red_rate_calc(year)
    
    if savefile:
        location='/data/brogalla/reduction_forcing/'
        
        for i in range(1,13):
            if i < 10:
                save_file(location+'red_y'+str(year)+'m0'+str(i)+'.nc',red_rateyear[i-1,:,:])
            else:
                save_file(location+'red_y'+str(year)+'m'+str(i)+'.nc',red_rateyear[i-1,:,:])
        
    
    return red_rateyear

##### Calculations

In [30]:
red_rate = red_year(2002, savefile=False)